In [9]:
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Step 1: Dataset Preparation
# Organize your dataset with labeled images (speed hump, plain road, pothole)
# Directory structure:
# - train
#   - Humps
#   - Plain
#   - pothole
# - test
#   - Humps
#   - Plain
#   - pothole

In [3]:
# Step 2: Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
# Step 3: Model Architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 classes

In [5]:
# Step 4: Split Dataset
train_dir = "C:/Users/KIIT/OneDrive/Documents/DS-1/My Dataset/train"
test_dir = "C:/Users/KIIT/OneDrive/Documents/DS-1/My Dataset/test"

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 1361 images belonging to 3 classes.
Found 102 images belonging to 3 classes.


In [6]:
# Step 5: Model Training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=25,  # Adjust the number of epochs as needed
    validation_data=test_generator
)

Epoch 1/25
 5/43 [==>...........................] - ETA: 2:15 - loss: 6.8735 - accuracy: 0.3375

C:\Users\KIIT\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


43/43 [==============================] - 142s 3s/step - loss: 1.5850 - accuracy: 0.5849 - val_loss: 2.6360 - val_accuracy: 0.3824
Epoch 2/25
43/43 [==============================] - 157s 4s/step - loss: 0.3465 - accuracy: 0.8677 - val_loss: 3.2545 - val_accuracy: 0.4902
Epoch 3/25
43/43 [==============================] - 138s 3s/step - loss: 0.2832 - accuracy: 0.9008 - val_loss: 3.7203 - val_accuracy: 0.4804
Epoch 4/25
43/43 [==============================] - 143s 3s/step - loss: 0.2883 - accuracy: 0.8876 - val_loss: 3.8639 - val_accuracy: 0.4902
Epoch 5/25
43/43 [==============================] - 141s 3s/step - loss: 0.2385 - accuracy: 0.9155 - val_loss: 4.1256 - val_accuracy: 0.5490
Epoch 6/25
43/43 [==============================] - 138s 3s/step - loss: 0.1925 - accuracy: 0.9302 - val_loss: 4.7853 - val_accuracy: 0.4608
Epoch 7/25
43/43 [==============================] - 151s 3s/step - loss: 0.2122 - accuracy: 0.9206 - val_loss: 4.1124 - val_accuracy: 0.4706
Epoch 8/25
43/43 [======

In [11]:
# Step 6: Real-time Video Processing
cap = cv2.VideoCapture("D:\speed_humps.mp4")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for the model
    img = cv2.resize(frame, (224, 224))
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize pixel values

    # Make predictions using the trained model
    predictions = model.predict(img)

    # Get the predicted class
    predicted_class_index = np.argmax(predictions)
    classes = ['Speed Hump', 'Plain Road', 'Pothole']
    predicted_class_label = classes[predicted_class_index]

    # Visualize the result on the frame
    cv2.putText(frame, f'Prediction: {predicted_class_label}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the result
    cv2.imshow('Object Recognition', frame)

    # Break the loop if 'ESC' key is pressed
    if cv2.waitKey(30) == 27:
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 160ms/step


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [12]:
# Save the model
model.save('C:/Users/KIIT/OneDrive/Documents/Research/Patent/Vehicle/CNN_model.h5')